# Plot Results from All Algorithm Runs

This notebook loads and plots results from all three algorithms (sgd_relu, alt_em_sgd, alt_em_closed_form) that were run sequentially on the same data.


In [ ]:
import json
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import glob


In [ ]:
# Configuration: Set the output directory path
# You can either specify a specific experiment directory or use the most recent one
outputs_dir = "outputs"

# Option 1: Use the most recent experiment
experiment_dirs = sorted(glob.glob(os.path.join(outputs_dir, "*")))
if experiment_dirs:
    experiment_dir = experiment_dirs[-1]  # Most recent
    print(f"Using experiment directory: {experiment_dir}")
else:
    raise ValueError(f"No experiment directories found in {outputs_dir}")

# Option 2: Or specify a specific directory
# experiment_dir = "outputs/your_experiment_name_20250101_120000"


In [ ]:
# Load data for all algorithms
algorithms = ["sgd_relu", "alt_em_sgd", "alt_em_closed_form"]
data = {}

for algo in algorithms:
    algo_dir = os.path.join(experiment_dir, algo)
    if not os.path.exists(algo_dir):
        print(f"Warning: Directory {algo_dir} not found, skipping {algo}")
        continue
    
    # Load training history
    history_path = os.path.join(algo_dir, "training_history.csv")
    if os.path.exists(history_path):
        data[algo] = {"history": pd.read_csv(history_path)}
    else:
        print(f"Warning: {history_path} not found")
        continue
    
    # Load final metrics
    metrics_path = os.path.join(algo_dir, "final_metrics.json")
    if os.path.exists(metrics_path):
        with open(metrics_path, 'r') as f:
            data[algo]["final_metrics"] = json.load(f)
    
    # Load losses
    losses_path = os.path.join(algo_dir, "losses.json")
    if os.path.exists(losses_path):
        with open(losses_path, 'r') as f:
            data[algo]["losses"] = json.load(f)

print(f"Loaded data for {len(data)} algorithms: {list(data.keys())}")


## Plot Training and Validation Losses


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot training losses
ax1 = axes[0]
for algo, algo_data in data.items():
    history = algo_data["history"]
    if "train_loss" in history.columns:
        x_key = "epoch" if "epoch" in history.columns else "cycle"
        ax1.plot(history[x_key], history["train_loss"], label=algo, marker='o', markersize=3)
ax1.set_xlabel(x_key.capitalize())
ax1.set_ylabel("Train Loss")
ax1.set_title("Training Loss Over Time")
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot validation losses
ax2 = axes[1]
for algo, algo_data in data.items():
    history = algo_data["history"]
    if "val_loss" in history.columns:
        x_key = "epoch" if "epoch" in history.columns else "cycle"
        ax2.plot(history[x_key], history["val_loss"], label=algo, marker='s', markersize=3)
ax2.set_xlabel(x_key.capitalize())
ax2.set_ylabel("Validation Loss")
ax2.set_title("Validation Loss Over Time")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## Plot Training and Test Accuracies


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot training accuracies
ax1 = axes[0]
for algo, algo_data in data.items():
    history = algo_data["history"]
    if "train_acc" in history.columns:
        x_key = "epoch" if "epoch" in history.columns else "cycle"
        ax1.plot(history[x_key], history["train_acc"], label=algo, marker='o', markersize=3)
ax1.set_xlabel(x_key.capitalize())
ax1.set_ylabel("Train Accuracy")
ax1.set_title("Training Accuracy Over Time")
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot validation accuracies
ax2 = axes[1]
for algo, algo_data in data.items():
    history = algo_data["history"]
    if "val_acc" in history.columns:
        x_key = "epoch" if "epoch" in history.columns else "cycle"
        ax2.plot(history[x_key], history["val_acc"], label=algo, marker='s', markersize=3)
ax2.set_xlabel(x_key.capitalize())
ax2.set_ylabel("Validation Accuracy")
ax2.set_title("Validation Accuracy Over Time")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## Plot Final Metrics Comparison


In [ ]:
# Extract final metrics
final_train_losses = []
final_test_losses = []
final_train_accs = []
final_test_accs = []
algo_names = []

for algo, algo_data in data.items():
    if "final_metrics" in algo_data:
        metrics = algo_data["final_metrics"]
        final_train_losses.append(metrics.get("train_loss", 0))
        final_test_losses.append(metrics.get("test_loss", 0))
        final_train_accs.append(metrics.get("train_acc", 0))
        final_test_accs.append(metrics.get("test_acc", 0))
        algo_names.append(algo)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Final train loss
axes[0, 0].bar(algo_names, final_train_losses, color='skyblue')
axes[0, 0].set_ylabel("Final Train Loss")
axes[0, 0].set_title("Final Training Loss Comparison")
axes[0, 0].grid(True, alpha=0.3, axis='y')

# Final test loss
axes[0, 1].bar(algo_names, final_test_losses, color='lightcoral')
axes[0, 1].set_ylabel("Final Test Loss")
axes[0, 1].set_title("Final Test Loss Comparison")
axes[0, 1].grid(True, alpha=0.3, axis='y')

# Final train accuracy
axes[1, 0].bar(algo_names, final_train_accs, color='lightgreen')
axes[1, 0].set_ylabel("Final Train Accuracy")
axes[1, 0].set_title("Final Training Accuracy Comparison")
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Final test accuracy
axes[1, 1].bar(algo_names, final_test_accs, color='plum')
axes[1, 1].set_ylabel("Final Test Accuracy")
axes[1, 1].set_title("Final Test Accuracy Comparison")
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


## Summary Table


In [ ]:
# Create a summary DataFrame
summary_data = []
for algo, algo_data in data.items():
    if "final_metrics" in algo_data:
        metrics = algo_data["final_metrics"]
        summary_data.append({
            "Algorithm": algo,
            "Train Loss": f"{metrics.get('train_loss', 0):.6f}",
            "Test Loss": f"{metrics.get('test_loss', 0):.6f}",
            "Train Acc": f"{metrics.get('train_acc', 0):.4f}",
            "Test Acc": f"{metrics.get('test_acc', 0):.4f}",
            "Val Loss": f"{metrics.get('val_loss', 0):.6f}",
            "Val Acc": f"{metrics.get('val_acc', 0):.4f}"
        })

if summary_data:
    summary_df = pd.DataFrame(summary_data)
    print("\nSummary of Final Metrics:")
    print(summary_df.to_string(index=False))
else:
    print("No summary data available.")
